## IMPORT

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from hypso import Hypso1, Hypso2
sys.path.append(os.path.abspath("D:/Hierarchical Unmixing Label"))
import machi.base as ac

## FUNCTIONS

In [ ]:
#Functions
def plot_cm_rgb_composite(cm, title="", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), height=1092, contrast_enhancement=False):
    """
    Create and plot an RGB composite image from a hyperspectral cube.
    
    Parameters:
    - cm: Input hyperspectral data cube or array (can be flattened (h*w, n_bands) or 3D (h, w, n_bands))
    - title: Optional title for the plot
    - red_band_index: Index of the band to use for red channel (default: 69)
    - green_band_index: Index of the band to use for green channel (default: 46)
    - blue_band_index: Index of the band to use for blue channel (default: 26)
    - aspect: Aspect ratio for the plot (default: 0.1)
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10))
    - height: Height of the image when reshaping from flattened data (default: 1092)
    
    Returns:
    - rgb_image: The processed RGB image
    """
    # Check if input is already a 3D cube or needs reshaping
    if len(cm.shape) == 2:  # Flattened data (h*w, n_bands)
        width = cm.shape[0] // height
        data_cube = cm.reshape(width, height, cm.shape[1])
        data_cube = np.transpose(data_cube, (0, 1, 2))  # Ensure correct orientation
    elif len(cm.shape) == 3:  # Already a cube (h, w, n_bands)
        data_cube = cm
    else:
        raise ValueError(f"Unexpected input shape: {cm.shape}. Expected 2D or 3D array.")
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)
    
    # Process data for better visualization
    # Replace NaN values with 0
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Rotate for proper orientation
    rgb_image = np.rot90(rgb_image)
    
    # Create and display the plot
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if title:
        plt.title(title)
    plt.imshow(rgb_image, aspect=aspect)
    plt.axis('off')  # Hide axes for cleaner visualization
    
    return rgb_image

def hypso_MACHI_Concatenate(hypso_file_path, cube_combined, saturated_combined, verbose=True, correction="L1D", hypso=1, spectra=112):

    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name}")

    correction_with_spectra = f"{correction}_{spectra}"
    
    # load and calibrate the hypso image, e.g. convert to radiance
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=True)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=True)

    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis = -1) #rawcube is not an attribute of the Hypso 2.0+ object  
    saturated = maxes > 35000
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)

    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    else:
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube

   # Extract spectral bands 
    if spectra==120:
        cube=np.array(data)
        c = cube.reshape(-1,spectra)
    else:
        spectra=112
        cube=np.array(data)[:,:,6:118]
        c = cube.reshape(-1,spectra)
    
    if verbose:
        print("saturated.shape: ",saturated.shape)
        print("cube.shape: ",cube.shape)
        print("c.shape: ",c.shape)

    T, S, objs = ac.atm_correction(c, solar=np.ones(spectra), verbose=False, tol=0.01, est_min_R=0.05)
    cube_norm = (cube - S) /T
    c_norm = cube_norm.reshape(-1,spectra)
    
    mins = [np.nanmin(c_norm[:,i]) for i in range(spectra)] #when saturation sets to nan, the min is not defined and the following line will set the min to nan
    cm_norm = c_norm - 0.95*np.array(mins)
    
    # Handle NaN values
    cm_norm = np.nan_to_num(cm_norm, nan=0.0)
    
    cm_combined = np.concatenate((cm_norm, cube_combined), axis=0)
    saturated_output = np.concatenate((saturated_flat, saturated_combined), axis=0)
    saturated_output = np.asarray(saturated_output, dtype=bool)
    
    if verbose:
        print("cube_norm.shape: ",cube_norm.shape)
        print("c_norm.shape: ",c_norm.shape)
        print("cm_norm.shape: ",cm_norm.shape)
        print("cm_combined.shape: ",cm_combined.shape)
        print("saturated_flat.shape: ",saturated_flat.shape)
        print("saturated_output.shape: ",saturated_output.shape)

    # Reshape cm back to cube shape for visualization
    cm_cube = cm_norm.reshape(cube.shape)
    # Use the plot_cm_rgb_composite function for visualization
    rgb=plot_cm_rgb_composite(
        cm_cube, 
        title=f"{image_name} ("+correction_with_spectra+"MACHI)",
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    # Save outputs
    output_dir = r'D:\Hierarchical Unmixing Label\hUH\images'
    plt.savefig(os.path.join(output_dir, f'{image_name}_rgb_composite_{correction_with_spectra}_MACHI.png'))
    np.save(os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}_MACHI.npy'), cm_norm)
    # np.save(os.path.join(output_dir, f'{image_name}_saturated.npy'), saturated_flat)
    if verbose:
        print(f"File saved to: {os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}_MACHI.npy')}")
        print("-"*50+'\n')
    return cm_combined, saturated_output, cm_norm, satobj

def hypso_Concatenate(hypso_file_path, cm_combined_input, saturated_combined_input, verbose=True, correction="L1A", hypso=1, spectra=120):
    """
    Process Hypso L1B data, normalize it, and concatenate with existing data.
    Uses a single, consistent normalization approach.
    
    Parameters:
    -----------
    hypso_file_path : str
        Path to the Hypso L1A file
    cm_combined_input : numpy.ndarray
        Existing combined cube data to append to
    saturated_combined_input : numpy.ndarray
        Existing combined saturation mask to append to
    verbose : bool, optional
        Whether to print processing information (default: True)
        
    Returns:
    --------
    tuple
        (combined data, combined saturation mask, normalized data, Hypso object)
    """
    # Extract image name from file path
    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name} using {correction}")
    
    # Load and calibrate the hypso image
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=True)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=True)
    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis=-1)
    saturated = maxes > 35000
    
    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    elif correction == "L1D":
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube
    else:
        print("correction valid, defaulting to L1B")
        correction="L1B"
        data = satobj.l1b_cube

    # Extract spectral bands 6-118
    if spectra==112:
        cube=np.array(data)[:,:,6:118]
        c = cube.reshape(-1,112)
    elif spectra==120:
        cube=np.array(data)
        c = cube.reshape(-1,120)
    # Print information about the image data values
    # Combine correction level with spectral band count to create a complete correction identifier
    correction_with_spectra = f"{correction}_{spectra}"
    
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)
    
    cm=c
    
    # Combine with existing data
    cm_combined = np.concatenate((cm, cm_combined_input), axis=0)
    saturated_combined = np.concatenate((saturated_flat, saturated_combined_input), axis=0)
    saturated_combined = np.asarray(saturated_combined, dtype=bool)

    # Reshape cm back to cube shape for visualization
    cm_cube = cm.reshape(cube.shape)
    # Use the plot_cm_rgb_composite function for visualization
    rgb=plot_cm_rgb_composite(
        cm_cube, 
        title=f"{image_name} "+correction_with_spectra,
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    
    # Save outputs
    output_dir = r'D:\Hierarchical Unmixing Label\hUH\images'
    # Fix the file path by properly joining the strings
    plt.savefig(os.path.join(output_dir, f'{image_name}_rgb_composite_{correction_with_spectra}.png'))
    np.save(os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}.npy'), cm)
    # np.save(os.path.join(output_dir, f'{image_name}_saturated.npy'), saturated_flat)
    print("-"*50+'\n')

    return cm_combined, saturated_combined, cm, satobj


## COMBINE IMAGES

In [ ]:
combined_L1D_MACHI= np.empty((0,112))
combined_L1A= np.empty((0,120))

combined_saturated = np.empty(0)

In [ ]:
combined_L1D_MACHI, combined_saturated, caspiansea1_2025_04_08, satobj_caspiansea1_2025_04_08 =hypso_MACHI_Concatenate(r"D:\Downloads\caspiansea1_2025-04-08T07-11-56Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)

In [ ]:
combined_L1D_MACHI, combined_saturated, yucatan2_2025_02_06_L1D,            satobj_yucatan2_2025_02_06          =hypso_MACHI_Concatenate(r"D:\Downloads\yucatan2_2025-02-06T16-01-18Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, kemigawa_2024_12_17_L1D,            satobj_kemigawa_2024_12_17          =hypso_MACHI_Concatenate(r"D:\Downloads\kemigawa_2024-12-17T01-01-32Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, chapala_2025_02_24_L1D,             satobj_chapala_2025_02_24           =hypso_MACHI_Concatenate(r"D:\Downloads\chapala_2025-02-24T16-52-47Z-l1a.nc",           combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, grizzlybay_2025_01_27_L1D,          satobj_grizzlybay_2025_01_27        =hypso_MACHI_Concatenate(r"D:\Downloads\grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, victoriaLand_2025_02_07_L1D,        satobj_victoriaLand_2025_02_07      =hypso_MACHI_Concatenate(r"D:\Downloads\victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, catala_2025_01_28_L1D,              satobj_catala_2025_01_28            =hypso_MACHI_Concatenate(r"D:\Downloads\catala_2025-01-28T19-17-32Z-l1a.nc",            combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, khnifiss_2025_02_12_L1D,            satobj_khnifiss_2025_02_12          =hypso_MACHI_Concatenate(r"D:\Downloads\khnifiss_2025-02-12T11-05-35Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, menindee_2025_02_18_L1D,            satobj_menindee_2025_02_18          =hypso_MACHI_Concatenate(r"D:\Downloads\menindee_2025-02-18T00-10-42Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, falklandsatlantic_2024_12_18_L1D,   satobj_falklandsatlantic_2024_12_18 =hypso_MACHI_Concatenate(r"D:\Downloads\falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, tampa_2024_11_12_L1D,               satobj_tampa_2024_11_12             =hypso_MACHI_Concatenate(r"D:\Downloads\tampa_2024-11-12T15-31-55Z-l1a.nc",             combined_L1D_MACHI, combined_saturated)

In [ ]:
combined_L1A, combined_saturated, yucatan2_2025_02_06_L1A,            satobj_yucatan2_2025_02_06          =hypso_Concatenate(r"D:\Downloads\yucatan2_2025-02-06T16-01-18Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, kemigawa_2024_12_17_L1A,            satobj_kemigawa_2024_12_17          =hypso_Concatenate(r"D:\Downloads\kemigawa_2024-12-17T01-01-32Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, chapala_2025_02_24_L1A,             satobj_chapala_2025_02_24           =hypso_Concatenate(r"D:\Downloads\chapala_2025-02-24T16-52-47Z-l1a.nc",           combined_L1A, combined_saturated)
combined_L1A, combined_saturated, grizzlybay_2025_01_27_L1A,          satobj_grizzlybay_2025_01_27        =hypso_Concatenate(r"D:\Downloads\grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        combined_L1A, combined_saturated)
combined_L1A, combined_saturated, victoriaLand_2025_02_07_L1A,        satobj_victoriaLand_2025_02_07      =hypso_Concatenate(r"D:\Downloads\victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      combined_L1A, combined_saturated)
combined_L1A, combined_saturated, catala_2025_01_28_L1A,              satobj_catala_2025_01_28            =hypso_Concatenate(r"D:\Downloads\catala_2025-01-28T19-17-32Z-l1a.nc",            combined_L1A, combined_saturated)
combined_L1A, combined_saturated, khnifiss_2025_02_12_L1A,            satobj_khnifiss_2025_02_12          =hypso_Concatenate(r"D:\Downloads\khnifiss_2025-02-12T11-05-35Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, menindee_2025_02_18_L1A,            satobj_menindee_2025_02_18          =hypso_Concatenate(r"D:\Downloads\menindee_2025-02-18T00-10-42Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, falklandsatlantic_2024_12_18_L1A,   satobj_falklandsatlantic_2024_12_18 =hypso_Concatenate(r"D:\Downloads\falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", combined_L1A, combined_saturated)
combined_L1A, combined_saturated, tampa_2024_11_12_L1A,               satobj_tampa_2024_11_12             =hypso_Concatenate(r"D:\Downloads\tampa_2024-11-12T15-31-55Z-l1a.nc",             combined_L1A, combined_saturated)

## SAVE

In [ ]:
np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy', combined_L1D_MACHI)
np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_saturated.npy', combined_saturated)

In [ ]:
np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1A_120.npy', combined_L1A)
# np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_saturated.npy', combined_saturated)

## LOAD

In [ ]:
combined_L1D_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy')
combined_saturated = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_saturated.npy')

In [ ]:
yucatan2_2025_02_06_L1D         = np.load(r'images\yucatan2_2025-02-06T16-01-18Z-l1a_cm_L1D_112_MACHI.npy')
kemigawa_2024_12_17_L1D         = np.load(r'images\kemigawa_2024-12-17T01-01-32Z-l1a_cm_L1D_112_MACHI.npy')
chapala_2025_02_24_L1D          = np.load(r'images\chapala_2025-02-24T16-52-47Z-l1a_cm_L1D_112_MACHI.npy')
grizzlybay_2025_01_27_L1D       = np.load(r'images\grizzlybay_2025-01-27T18-19-56Z-l1a_cm_L1D_112_MACHI.npy')
victoriaLand_2025_02_07_L1D     = np.load(r'images\victoriaLand_2025-02-07T20-35-33Z-l1a_cm_L1D_112_MACHI.npy')
catala_2025_01_28_L1D           = np.load(r'images\catala_2025-01-28T19-17-32Z-l1a_cm_L1D_112_MACHI.npy')
khnifiss_2025_02_12_L1D         = np.load(r'images\khnifiss_2025-02-12T11-05-35Z-l1a_cm_L1D_112_MACHI.npy')
menindee_2025_02_18_L1D         = np.load(r'images\menindee_2025-02-18T00-10-42Z-l1a_cm_L1D_112_MACHI.npy')
falklandsatlantic_2024_12_18_L1D= np.load(r'images\falklandsatlantic_2024-12-18T13-25-18Z-l1a_cm_L1D_112_MACHI.npy')
tampa_2024_11_12_L1D            = np.load(r'images\tampa_2024-11-12T15-31-55Z-l1a_cm_L1D_112_MACHI.npy')

## PLOT

In [ ]:
rgb_maci_combined = plot_cm_rgb_composite(combined_L1D_MACHI, "combined_L1D_MACHI")

In [ ]:
rgb_yucatan2_2025_02_06_L1D = plot_cm_rgb_composite(yucatan2_2025_02_06_L1D, "yucatan2_2025_02_06")
rgb_kemigawa_2024_12_17_L1D = plot_cm_rgb_composite(kemigawa_2024_12_17_L1D, "kemigawa_2024_12_17")
rgb_chapala_2025_02_24_L1D = plot_cm_rgb_composite(chapala_2025_02_24_L1D, "chapala_2025_02_24")
rgb_grizzlybay_2025_01_27_L1D = plot_cm_rgb_composite(grizzlybay_2025_01_27_L1D, "grizzlybay_2025_01_27")
rgb_victoriaLand_2025_02_07_L1D = plot_cm_rgb_composite(victoriaLand_2025_02_07_L1D, "victoriaLand_2025_02_07")
rgb_catala_2025_01_28_L1D = plot_cm_rgb_composite(catala_2025_01_28_L1D, "catala_2025_01_28")
rgb_khnifiss_2025_02_12_L1D = plot_cm_rgb_composite(khnifiss_2025_02_12_L1D, "khnifiss_2025_02_12")
rgb_menindee_2025_02_18_L1D = plot_cm_rgb_composite(menindee_2025_02_18_L1D, "menindee_2025_02_18")
rgb_falklandsatlantic_2024_12_18_L1D = plot_cm_rgb_composite(falklandsatlantic_2024_12_18_L1D, "falklandsatlantic_2024_12_18")
rgb_tampa_2024_11_12_L1D = plot_cm_rgb_composite(tampa_2024_11_12_L1D, "tampa_2024_11_12")
#rgb_aquawatchmoreton_2024_09_02 = plot_cm_rgb_composite(aquawatchmoreton_2024_09_02, "aquawatchmoreton_2024_09_02") #Edge of Space
# rgb_mjosa_2025_02_11 = plot_cm_rgb_composite(mjosa_2025_02_11, "mjosa_2025_02_11") # directly into space

In [ ]:
rgb_yucatan2_2025_02_06_L1A = plot_cm_rgb_composite(yucatan2_2025_02_06_L1A, "yucatan2_2025_02_06")
rgb_kemigawa_2024_12_17_L1A = plot_cm_rgb_composite(kemigawa_2024_12_17_L1A, "kemigawa_2024_12_17")
rgb_chapala_2025_02_24_L1A = plot_cm_rgb_composite(chapala_2025_02_24_L1A, "chapala_2025_02_24")
rgb_grizzlybay_2025_01_27_L1A = plot_cm_rgb_composite(grizzlybay_2025_01_27_L1A, "grizzlybay_2025_01_27")
rgb_victoriaLand_2025_02_07_L1A = plot_cm_rgb_composite(victoriaLand_2025_02_07_L1A, "victoriaLand_2025_02_07")
rgb_catala_2025_01_28_L1A = plot_cm_rgb_composite(catala_2025_01_28_L1A, "catala_2025_01_28")
rgb_khnifiss_2025_02_12_L1A = plot_cm_rgb_composite(khnifiss_2025_02_12_L1A, "khnifiss_2025_02_12")
rgb_menindee_2025_02_18_L1A = plot_cm_rgb_composite(menindee_2025_02_18_L1A, "menindee_2025_02_18")
rgb_falklandsatlantic_2024_12_18_L1A = plot_cm_rgb_composite(falklandsatlantic_2024_12_18_L1A, "falklandsatlantic_2024_12_18")
rgb_tampa_2024_11_12_L1A = plot_cm_rgb_composite(tampa_2024_11_12_L1A, "tampa_2024_11_12")

## DEBUG

In [ ]:
#empty